<a href="https://colab.research.google.com/github/Karine-Moussa/PANGO-Genomic-Conversions/blob/main/COVID_19_PANGO_All_Variants_Genomic_Conversion_Chart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PANGO lineages of concern/note webscraper

Pulls SNPs from all lineages of concern/note from https://cov-lineages.org/global_report.html and converts to genome coordinates.

## Instructions
- "Runtime > Run all" to use

- To download the output table, go to the left tab and select the *files* icon > right-click on `snpaa.csv` > select *download*

 <img width="200" alt="image" src="https://i.imgur.com/pNNC5YU.png">


## Webscraper

In [ ]:
import requests as rq
import re
from bs4 import BeautifulSoup as bs
import pandas as pd

In [ ]:
#Grabs HTML of main covid lineage page and makes soup object for parsing
main_url = 'https://cov-lineages.org/global_report.html'
main_html = rq.get(main_url)
main_soup = bs(main_html.content, 'html.parser')

In [ ]:
#Base URL to be added to any relative links
base_url = 'https://cov-lineages.org'

#Find all <a> tags
all_a = main_soup.find_all('a')

#Create a list with URL to each lineage of concern
url_list = []
for i in all_a: 
  a = i.get('href')
  if './global_report' in a:
    url_list.append(base_url + a)

In [ ]:
#Grabs HTML from each URL from above and parses for SNPs
snp_list = []
for i in url_list:
  a = rq.get(i)
  b = bs(a.content, 'html.parser')
  c = b.find(text='Defining SNPs').find_next('td')
  snp_list.append(i[40:-5] + ' ' + c.get_text())

In [ ]:
#Convert list of SNPs into a df. There's probably a neater way to do this...
variants = []
snps = []
for i in snp_list:
  a = i.split()
  n = len(a) - 1
  v = a[0] + ' '
  b = (v * n).split()
  c = a[1:]
  variants += b
  snps += c
data = {'variant':variants, 'snps':snps}

snpdf = pd.DataFrame(data)
snpdf['mutation_class'] = snpdf['snps'].str.extract('([a-z A-Z]+)')
snpdf['formatted'] = snpdf['snps'].str.extract(':([a-z 0-9]+:[a-z][0-9]+[a-z\*]+)', flags=re.IGNORECASE)

##Set up

In [ ]:
# Install python packages
!pip -q install pyfaidx # -q suppresses pip output message

In [ ]:
# Import python libraries
import os.path
from os import path
import pandas as pd
from pyfaidx import Fasta

In [ ]:
# Get Genome
if path.exists("GCF_009858895.2_ASM985889v3_genomic.fna") == False:
  !wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/009/858/895/GCF_009858895.2_ASM985889v3/GCF_009858895.2_ASM985889v3_genomic.fna.gz
  !gunzip GCF_009858895.2_ASM985889v3_genomic.fna.gz

In [ ]:
# Get genome gff 
if path.exists("GCF_009858895.2_ASM985889v3_genomic.gff") == False:
  !wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/009/858/895/GCF_009858895.2_ASM985889v3/GCF_009858895.2_ASM985889v3_genomic.gff.gz
  !gunzip GCF_009858895.2_ASM985889v3_genomic.gff

In [ ]:
# Load genome, grab DNA like genome["NC_045512.2"][start:end]
genome = Fasta('GCF_009858895.2_ASM985889v3_genomic.fna')

In [ ]:
# Load gff as pandas data frame
df = pd.read_csv("GCF_009858895.2_ASM985889v3_genomic.gff", comment="#", sep="\t", header=None)

In [ ]:
# Collect GFF information on gene
df = df[df[2] == "gene"]
df[8] = df[8].apply(lambda line: [n.lstrip("gene=").lower() for n in line.split(";") if n.startswith("gene=")][0])
# Manually add a row for orf1a (266..13483 taken from https://www.ncbi.nlm.nih.gov/nuccore/NC_045512.2) 
orf1a = pd.DataFrame(pd.DataFrame([['NC_045512.2', 'RefSeq','gene',266,13483,'.','+','.','orf1a']],)) 
df = df.append(orf1a, ignore_index=True)
genes_list = list(df[8])

## Conversion

In [ ]:
codon_table = {"TTT": "F", "TTC": "F", "TTA": "L", "TTG": "L",
               "CTT": "L", "CTC": "L", "CTA": "L", "CTG": "L",
               "ATT": "I", "ATC": "I", "ATA": "I", "ATG": "M",
               "GTT": "V", "GTC": "V", "GTA": "V", "GTG": "V",
               "TCT": "S", "TCC": "S", "TCA": "S", "TCG": "S",
               "CCT": "P", "CCC": "P", "CCA": "P", "CCG": "P",
               "ACT": "T", "ACC": "T", "ACA": "T", "ACG": "T",
               "GCT": "A", "GCC": "A", "GCA": "A", "GCG": "A",
               "TAT": "Y", "TAC": "Y", "TAA": "*", "TAG": "*",
               "CAT": "H", "CAC": "H", "CAA": "Q", "CAG": "Q",
               "AAT": "N", "AAC": "N", "AAA": "K", "AAG": "K",
               "GAT": "D", "GAC": "D", "GAA": "E", "GAG": "E",
               "TGT": "C", "TGC": "C", "TGA": "*", "TGG": "W",
               "CGT": "R", "CGC": "R", "CGA": "R", "CGG": "R",
               "AGT": "S", "AGC": "S", "AGA": "R", "AGG": "R",
               "GGT": "G", "GGC": "G", "GGA": "G", "GGG": "G"}

In [ ]:
# Create dna -> aa translate function
def translate(seq):
    p = ""
    for loc in range(0, len(seq), 3):
        p += codon_table[seq[loc:loc+3]]
    return p

In [ ]:
def mutant_parse(mutation):
  gene_name, before, loc, after = re.match("([a-z 0-9]+):([a-z]+)([0-9]+)([a-z\*]+)", mutation, flags=re.IGNORECASE).groups()
  loc = int(loc)
  return gene_name, before, loc, after

In [ ]:
def get_gene_start(x):
  if x.lower() in genes_list:
    start = int(df[df[8]==x.lower()][3].values[0])
  else:
    start = int()
  return start

def get_gene_end(x):
  if x.lower() in genes_list:
    end = int(df[df[8]==x.lower()][4].values[0])
  else:
    end = int()
  return end

In [ ]:
#create new dataframe w/o del mutations and parse mutation
snpaa = snpdf[~snpdf['mutation_class'].str.contains("del")]
snpaa[['gene_name','before','loc','after']] = snpaa.apply(
    lambda x: pd.Series(mutant_parse(x['formatted'])), axis=1)

#add gene start and end to dataframe
snpaa['gene_start'] = snpaa['gene_name'].apply(get_gene_start)
snpaa['gene_end'] = snpaa['gene_name'].apply(get_gene_end)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [ ]:
### Locate the amino acid in the genome
snpaa['mut_start'] = (snpaa['gene_start']-1 + 3*(snpaa['loc']-1))
snpaa['mut_end'] = (snpaa['gene_start']-1 + 3*(snpaa['loc']-1) + 3)

### Translate the amino acid in the genome
snpaa['trans_aa'] = snpaa.apply(
    lambda x: translate(str(genome["NC_045512.2"][x['mut_start']:x['mut_end']])), axis =1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [ ]:
# Add verification column
snpaa['pass'] = (snpaa['before'] == snpaa['trans_aa'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# Save output to csv
snpaa.to_csv('snpaa.csv')

## Output

**Note: The mut_start is the 0 based coordinate of the first nucleotide in the codon**

In [ ]:
snpaa

,variant,snps,mutation_class,formatted,gene_name,before,loc,after,gene_start,gene_end,mut_start,mut_end,trans_aa,pass
0,B.1.1.7,aa:orf1ab:T1001I,aa,orf1ab:T1001I,orf1ab,T,1001,I,266,21555,3265,3268,T,True
1,B.1.1.7,aa:orf1ab:A1708D,aa,orf1ab:A1708D,orf1ab,A,1708,D,266,21555,5386,5389,A,True
2,B.1.1.7,aa:orf1ab:I2230T,aa,orf1ab:I2230T,orf1ab,I,2230,T,266,21555,6952,6955,I,True
6,B.1.1.7,aa:S:N501Y,aa,S:N501Y,S,N,501,Y,21563,25384,23062,23065,N,True
7,B.1.1.7,aa:S:A570D,aa,S:A570D,S,A,570,D,21563,25384,23269,23272,A,True
8,B.1.1.7,aa:S:P681H,aa,S:P681H,S,P,681,H,21563,25384,23602,23605,P,True
9,B.1.1.7,aa:S:T716I,aa,S:T716I,S,T,716,I,21563,25384,23707,23710,T,True
10,B.1.1.7,aa:S:S982A,aa,S:S982A,S,S,982,A,21563,25384,24505,24508,S,True
11,B.1.1.7,aa:S:D1118H,aa,S:D1118H,S,D,1118,H,21563,25384,24913,24916,D,True
12,B.1.1.7,aa:Orf8:Q27*,aa,Orf8:Q27*,Orf8,Q,27,*,27894,28259,27971,27974,Q,True
